repo https://github.com/AlePaff/FIUBA-knowledge

In [1]:
import pandas as pd
import numpy as np
import re

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1xj78CNKOvswnTZ7_wIfFr-v9Pt3mV5-C'})
downloaded.GetContentFile('con links cargados.csv')

textos=pd.read_csv('con links cargados.csv')

##Analisis exploratorio

In [2]:
textos.groupby("asignaturasDeptos").count().sort_values(by=['asignaturas'], ascending = False)

# df.nunique().to_frame().T

,asignaturas,planificaciones
asignaturasDeptos,,
https://www.fi.uba.ar/institucional/departamentos/electronica/asignaturas,212,212
https://www.fi.uba.ar/institucional/departamentos/computacion/asignaturas,115,115
https://www.fi.uba.ar/institucional/departamentos/gestion/asignaturas,98,98
https://www.fi.uba.ar/institucional/departamentos/energia/asignaturas,94,94
https://www.fi.uba.ar/institucional/departamentos/ingenieria-mecanica/asignaturas,85,85
https://www.fi.uba.ar/institucional/departamentos/ingenieria-quimica/asignaturas,48,46
https://www.fi.uba.ar/institucional/departamentos/agrimensura/asignaturas,47,47
https://www.fi.uba.ar/institucional/departamentos/tecnologia-industrial/asignaturas,45,45
https://www.fi.uba.ar/institucional/departamentos/departamento-de-construcciones-y-estructuras/asignaturas,43,43


In [59]:
textos["planificaciones"]

0      https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf
1      https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf
2      https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf
3      https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf
4      https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf
                             ...                        
978    https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf
979    https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf
980    https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf
981    https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf
982    https://cms.fi.uba.ar/uploads/8814_213fffa227.pdf
Name: planificaciones, Length: 983, dtype: object

In [3]:
#asignaturas que NO tienen un link a pdf válido (el ~ sirve para negar)
textosNoValidos = textos[~textos["planificaciones"].str.contains("cms.fi.uba.ar", na=False)]
textosNoValidos.shape

(29, 3)

In [4]:
textos = textos[textos["planificaciones"].str.contains("cms.fi.uba.ar", na=False)]

##PDF's

###pdfPlumber

In [5]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import urllib3
import pdfplumber
import io

# https://github.com/jsvine/pdfplumber
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [7]:
#https://stackoverflow.com/questions/62075033/read-pdf-from-url-to-memory-omitting-saving-file-to-local-file

#extrae todo el texto, de un enlace a un pdf (dependiendo del contenido puede tardar entre 0.6 seg a 1 seg)
def extract_pdf_by_url(url):
    http = urllib3.PoolManager()
    temp = io.BytesIO()
    temp.write(http.request("GET", url).data)
    all_text = ''
    with pdfplumber.open(temp) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            all_text = all_text + '\n' + single_page_text
    return all_text

In [8]:
#dado un pdf filtra la sección desde "objetivos" hasta "bibliografía"

#en caso de querer separarlos en más categorías guardar en sublistas
def extraerInfo(pdfActual):
  titulos = ["OBJETIVOS", "CONTENIDOS MÍNIMOS", "PROGRAMA SINTÉTICO", "PROGRAMA ANALÍTICO", "BIBLIOGRAFÍA"]

  infoGuardada = []
  guardar = False
  for palabra in pdfActual.splitlines():    #https://stackoverflow.com/questions/24237524/how-to-split-a-python-string-on-new-line-characters
    if(palabra == titulos[0]):
      guardar = True
    # else:
    #   print("Error al inicio en:", )
    if(palabra == titulos[-1]):
      guardar = False
    if guardar:
      infoGuardada.append(palabra)

  # print("\n". join(infoGuardada))
  return ("\n". join(infoGuardada))

In [9]:
textos

,asignaturasDeptos,asignaturas,planificaciones
0,https://www.fi.uba.ar/institucional/departamen...,70.00 Tesis,https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf
1,https://www.fi.uba.ar/institucional/departamen...,70.01 Geometría Proyectiva,https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf
2,https://www.fi.uba.ar/institucional/departamen...,70.02 Geometría Descriptiva,https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf
3,https://www.fi.uba.ar/institucional/departamen...,70.03 Medios de Representación A,https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf
4,https://www.fi.uba.ar/institucional/departamen...,70.04 Dibujo Topográfico,https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf
...,...,...,...
978,https://www.fi.uba.ar/institucional/departamen...,88.10 Caminos,https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf
979,https://www.fi.uba.ar/institucional/departamen...,88.11 Aeropuertos A,https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf
980,https://www.fi.uba.ar/institucional/departamen...,88.12 Ferrocarriles A,https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf
981,https://www.fi.uba.ar/institucional/departamen...,88.13 Puertos y Vías Navegables A,https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf


In [10]:
# creo una nueva columna
pd.options.mode.chained_assignment = None
textos["contenido"] = np.nan

textos.head(3)

,asignaturasDeptos,asignaturas,planificaciones,contenido
0,https://www.fi.uba.ar/institucional/departamen...,70.00 Tesis,https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf,NaN
1,https://www.fi.uba.ar/institucional/departamen...,70.01 Geometría Proyectiva,https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf,NaN
2,https://www.fi.uba.ar/institucional/departamen...,70.02 Geometría Descriptiva,https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf,NaN


In [130]:
#esta celda tarda aprox. 20 mins en ejecutarse
#añade a cada columna la información extraida anteriormente
total = textos.shape[0]
for i, link in enumerate(textos["planificaciones"]):     #enumerate devuelve el indice (un nro.) y lo que se itera
  pdfActual = extract_pdf_by_url(link)
  textos.iloc[i, 3] = extraerInfo(pdfActual)
  print(i+1,"/",total)

1 / 954
2 / 954
3 / 954
4 / 954
5 / 954
6 / 954
7 / 954
8 / 954
9 / 954
10 / 954
11 / 954
12 / 954
13 / 954
14 / 954
15 / 954
16 / 954
17 / 954
18 / 954
19 / 954
20 / 954
21 / 954
22 / 954
23 / 954
24 / 954
25 / 954
26 / 954
27 / 954
28 / 954
29 / 954
30 / 954
31 / 954
32 / 954
33 / 954
34 / 954
35 / 954
36 / 954
37 / 954
38 / 954
39 / 954
40 / 954
41 / 954
42 / 954
43 / 954
44 / 954
45 / 954
46 / 954
47 / 954
48 / 954
49 / 954
50 / 954
51 / 954
52 / 954
53 / 954
54 / 954
55 / 954
56 / 954
57 / 954
58 / 954
59 / 954
60 / 954
61 / 954
62 / 954
63 / 954
64 / 954
65 / 954
66 / 954
67 / 954
68 / 954
69 / 954
70 / 954
71 / 954
72 / 954
73 / 954
74 / 954
75 / 954
76 / 954
77 / 954
78 / 954
79 / 954
80 / 954
81 / 954
82 / 954
83 / 954
84 / 954
85 / 954
86 / 954
87 / 954
88 / 954
89 / 954
90 / 954
91 / 954
92 / 954
93 / 954
94 / 954
95 / 954
96 / 954
97 / 954
98 / 954
99 / 954
100 / 954
101 / 954
102 / 954
103 / 954
104 / 954
105 / 954
106 / 954
107 / 954
108 / 954
109 / 954
110 / 954
111 / 95

In [132]:
textos.head(3)

,asignaturasDeptos,asignaturas,planificaciones,contenido,depto,deptoNro,codMateria
0,https://www.fi.uba.ar/institucional/departamen...,70.00 Tesis,https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf,OBJETIVOS\nCONTENIDOS MÍNIMOS\n-\nPROGRAMA SIN...,agrimensura,70,7000
1,https://www.fi.uba.ar/institucional/departamen...,70.01 Geometría Proyectiva,https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf,OBJETIVOS\nCONTENIDOS MÍNIMOS\n-\nPROGRAMA SIN...,agrimensura,70,7001
2,https://www.fi.uba.ar/institucional/departamen...,70.02 Geometría Descriptiva,https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf,OBJETIVOS\nRepresentar sobre un plano formas t...,agrimensura,70,7002


In [131]:
textos.nunique().to_frame()

,0
asignaturasDeptos,18
asignaturas,954
planificaciones,954
contenido,803
depto,18
deptoNro,34
codMateria,895


### separar por departamentos

In [133]:
# https://stackoverflow.com/questions/36108377/how-to-use-the-split-function-on-every-row-in-a-dataframe-in-python
f = lambda x: x["asignaturasDeptos"].split("/")[5]
nroDepto = lambda x: x["asignaturas"].split(".")[0]
codMateria = lambda x: (x["asignaturas"].split(" ")[0]).replace(".","")   #toma 70.20 y lo convierte en 7020

textos["depto"] = textos.apply(f, axis=1)
textos["deptoNro"] = textos.apply(nroDepto, axis=1)
textos["codMateria"] = textos.apply(codMateria, axis=1)
textos.head(3)

,asignaturasDeptos,asignaturas,planificaciones,contenido,depto,deptoNro,codMateria
0,https://www.fi.uba.ar/institucional/departamen...,70.00 Tesis,https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf,OBJETIVOS\nCONTENIDOS MÍNIMOS\n-\nPROGRAMA SIN...,agrimensura,70,7000
1,https://www.fi.uba.ar/institucional/departamen...,70.01 Geometría Proyectiva,https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf,OBJETIVOS\nCONTENIDOS MÍNIMOS\n-\nPROGRAMA SIN...,agrimensura,70,7001
2,https://www.fi.uba.ar/institucional/departamen...,70.02 Geometría Descriptiva,https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf,OBJETIVOS\nRepresentar sobre un plano formas t...,agrimensura,70,7002


In [134]:
#efectivamente son 18 deptos
deptos = list(textos["depto"].unique())
print(deptos)
print("cant. de deptos: ", len(deptos))

#aunque hay algunos que no existen
# len(list(textos["deptoNro"].unique()))     #->da 34, y debería dar 18

['agrimensura', 'computacion', 'departamento-de-construcciones-y-estructuras', 'electronica', 'energia', 'estabilidad', 'fisica', 'gestion', 'hidraulica', 'idiomas', 'ingenieria-mecanica', 'ingenieria-naval', 'ingenieria-quimica', 'matematica', 'quimica', 'seguridad-del-trabajo-y-ambiente', 'tecnologia-industrial', 'transporte']
cant. de deptos:  18


#####departamentos


70.XX - Agrimensura (+)
  
75.XX - Computación (+)
  
74.XX - Construcciones y Estructuras (+)
  
66.XX - Electrónica (+)
 
65.XX - Electrotecnia (+)

64.XX - Estabilidad (+)

62.XX - Física (+)

71.XX - Gestión (+)

69.XX - Hidráulica (+)

78.XX - Idiomas (+)

67.XX - Ingeniería Mecánica (+)

73.XX - Ingeniería Naval (+)

76.XX - Ingeniería Química (+)

61.XX - Matemática (+)

63.XX - Química (+)

77.XX - Seguridad del Trabajo y Ambiente (+)

72.XX - Tecnología Industrial (+)

68.XX - Transporte (+)

http://back.fi.uba.ar/es/node/792

#####alternativa hardcodeada

In [19]:
# agrimensura = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/agrimensura/asignaturas"]
# computacion = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/computacion/asignaturas"]
# departamentodeconstruccionesyestructuras = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/departamento-de-construcciones-y-estructuras/asignaturas"]
# electronica = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/electronica/asignaturas"]
# energia = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/energia/asignaturas"]
# estabilidad = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/estabilidad/asignaturas"]
# fisica = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/fisica/asignaturas"]
# gestion = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/gestion/asignaturas"]
# hidraulica = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/hidraulica/asignaturas"]
# idiomas = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/idiomas/asignaturas"]
# ingenieriamecanica = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/ingenieria-mecanica/asignaturas"]
# ingenierianaval = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/ingenieria-naval/asignaturas"]
# ingenieriaquimica = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/ingenieria-quimica/asignaturas"]
# matematica = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/matematica/asignaturas"]
# quimica = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/quimica/asignaturas"]
# seguridaddeltrabajoyambiente = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/seguridad-del-trabajo-y-ambiente/asignaturas"]
# tecnologiaindustrial = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/tecnologia-industrial/asignaturas"]
# transporte = textos[textos["asignaturasDeptos"] == "https://www.fi.uba.ar/institucional/departamentos/transporte/asignaturas"]

# deptos = [agrimensura,computacion,departamentodeconstruccionesyestructuras,electronica,energia,estabilidad,fisica,gestion,hidraulica,idiomas,ingenieriamecanica,ingenierianaval,ingenieriaquimica,matematica,quimica,seguridaddeltrabajoyambiente,tecnologiaindustrial,transporte]
# deptos

####formato json

In [136]:
textos

,asignaturasDeptos,asignaturas,planificaciones,contenido,depto,deptoNro,codMateria
0,https://www.fi.uba.ar/institucional/departamen...,70.00 Tesis,https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf,OBJETIVOS\nCONTENIDOS MÍNIMOS\n-\nPROGRAMA SIN...,agrimensura,70,7000
1,https://www.fi.uba.ar/institucional/departamen...,70.01 Geometría Proyectiva,https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf,OBJETIVOS\nCONTENIDOS MÍNIMOS\n-\nPROGRAMA SIN...,agrimensura,70,7001
2,https://www.fi.uba.ar/institucional/departamen...,70.02 Geometría Descriptiva,https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf,OBJETIVOS\nRepresentar sobre un plano formas t...,agrimensura,70,7002
3,https://www.fi.uba.ar/institucional/departamen...,70.03 Medios de Representación A,https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf,OBJETIVOS\n1-GENERALES(cid:13)\n(cid:13)\n1.1-...,agrimensura,70,7003
4,https://www.fi.uba.ar/institucional/departamen...,70.04 Dibujo Topográfico,https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf,OBJETIVOS\nIntroducir al alumno en las técnica...,agrimensura,70,7004
...,...,...,...,...,...,...,...
978,https://www.fi.uba.ar/institucional/departamen...,88.10 Caminos,https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf,OBJETIVOS\nLa asignatura se centra en el trans...,transporte,88,8810
979,https://www.fi.uba.ar/institucional/departamen...,88.11 Aeropuertos A,https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf,OBJETIVOS\nIntroducir al estudiante de grado e...,transporte,88,8811
980,https://www.fi.uba.ar/institucional/departamen...,88.12 Ferrocarriles A,https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf,OBJETIVOS\nEl objetivo primordial es el de pro...,transporte,88,8812
981,https://www.fi.uba.ar/institucional/departamen...,88.13 Puertos y Vías Navegables A,https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf,OBJETIVOS\nProporcionar a los alumnos un conoc...,transporte,88,8813


In [159]:
# elimina lineas del estilo: "7003 - Medios de Representación A PLANIFICACIONES Actualización: 1ºC/2018",   "7004 - Dibujo Topográfico PLANIFICACIONES Actualización: 1ºC/2018"
# para un filtrado mejor se puede recortar el header y footer del pdf https://stackoverflow.com/questions/66900539/how-to-stop-pdfplumber-from-reading-the-header-of-every-pages
def eliminarLineaExtra(x):
  elem = x.split("\n")
  for i in elem:
    if ("PLANIFICACIONES") in i:
      elem.remove(i)
  return "\n".join(elem)

#limpieza
textos["contenido"] = textos["contenido"].apply(lambda x: re.sub(r"\(cid:[0-9]{1,}\)", "", str(x)))
textos["contenido"] = textos["contenido"].apply(lambda x: re.sub(r"[0-9] de [0-9]", "", str(x)))
textos["contenido"] = textos["contenido"].apply(lambda x: eliminarLineaExtra(x))

In [160]:
textos[["asignaturas", "planificaciones", "contenido", "depto", "codMateria"]].to_json(r"materias.json", orient = "records")

---

####alternativa si separaba por deptos para formato markdown

In [ ]:
#formato markdown
mdDeptos = []

for depto in deptos:
  markdownFile = []
  for i in range(depto.shape[0]):
    markdownFile.append("### "+depto.iloc[i, 1])  #nombre materia
    markdownFile.append("\n\n[link al pdf]("+depto.iloc[i, 2]+")\n\n\n")      #link al pdf
    markdownFile.append(depto.iloc[i, 3])   #planificacion materia
    markdownFile.append("\n\n")
  mdDeptos.append(markdownFile)

In [ ]:
#limpieza

for i, mdDepto in enumerate(mdDeptos):
  mdPlain = []
  mdPlain = "".join(markdownFile)
  mdPlain = re.sub(r"\(cid:[0-9]{1,}\)", "", mdPlain)     #me reemplaza (cid:9), (cid:13), etc
  mdPlain = re.sub(r"[0-9] de [0-9]", "", mdPlain)     #me reemplaza los textos 2 de 9, 5 de 9, etc
  mdPlain = mdPlain.replace("\n-\n", "\n")          #para que el formato markdown no flashee
  mdPlain = mdPlain.replace("\n", "\n\n")     #para que markdown no ponga todo pegado sin saltos de linea 
  mdDeptos[i] = mdPlain

In [ ]:
!mkdir "deptos"

In [ ]:

stringDeptos = ["agrimensura.md","computacion.md","departamentodeconstruccionesyestructuras.md","electronica.md","energia.md","estabilidad.md","fisica.md","gestion.md","hidraulica.md","idiomas.md","ingenieriamecanica.md","ingenierianaval.md","ingenieriaquimica.md","matematica.md","quimica.md","seguridaddeltrabajoyambiente.md","tecnologiaindustrial.md","transporte.md"]
for i, mdDepto in enumerate(mdDeptos):
  with open("deptos/"+stringDeptos[i], "w") as text_file:
      text_file.write(mdDepto)

In [ ]:
!zip -r deptos.zip "deptos/"

#####tests

In [ ]:
pdfs = ["https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf","https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf","https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf","https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf","https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf","https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf","https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf","https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf","https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf","https://cms.fi.uba.ar/uploads/8814_213fffa227.pdf"]

In [ ]:
sample5.iloc[0, 1]

'70.00 Tesis'

###todo en un archivo markdown

In [ ]:
#formato markdown
markdownFile = []

for i in range(textos.shape[0]):
  markdownFile.append("### "+textos.iloc[i, 1])  #nombre materia
  markdownFile.append("\n\n[link al pdf]("+textos.iloc[i, 2]+")\n\n\n")      #link al pdf
  markdownFile.append(textos.iloc[i, 3])   #planificacion materia
  markdownFile.append("\n\n")

In [ ]:
#limpieza
plainMarkdown = "".join(markdownFile)
plainMarkdown = re.sub(r"\(cid:[0-9]{1,}\)", "", plainMarkdown)     #me reemplaza (cid:9), (cid:13), etc
plainMarkdown = re.sub(r"[0-9] de [0-9]", "", plainMarkdown)     #me reemplaza los textos 2 de 9, 5 de 9, etc
plainMarkdown = plainMarkdown.replace("\n-\n", "\n")          #para que el formato markdown no flashee
# plainMarkdown = plainMarkdown.replace("\n", "\n\n")     #para que markdown no ponga todo pegado sin saltos de linea 

In [ ]:
print(plainMarkdown)

In [ ]:
with open("contenidos.md", "w") as text_file:
    text_file.write(plainMarkdown)

el formato no es modificable (solo desde notapad++, cambiarlo a html permite añadir spoilers e índices, sin embargo sigue sin ser un formato modificable) sin embargo **es un proyecto sencillo**

https://dillinger.io/ para testear markdown de github


###Tika


In [ ]:
!pip install tika

In [ ]:
from tika import parser

http = urllib3.PoolManager()
temp = io.BytesIO()
temp.write(http.request("GET", url).data)

rawText = parser.from_file(temp)

rawList = rawText['content'].splitlines()


###PyPDF2


In [ ]:
!pip install PyPDF2

In [ ]:
import requests, PyPDF2
from io import BytesIO

url = 'https://cms.fi.uba.ar/uploads/7008_004dc77220.pdf'
response = requests.get(url)
my_raw_data = response.content

with BytesIO(my_raw_data) as data:
    read_pdf = PyPDF2.PdfFileReader(data)

    for page in range(read_pdf.getNumPages()):
        print(read_pdf.getPage(page).extractText())


Planificaciones7008 - Topografía IDocente responsable: PRADELLI ALBERTO1 de 57008 - Topografía IPLANIFICACIONESActualización: 1ºC/2016
OBJETIVOS1- Integrar conocimientos básicos de matemática, física, geometría y dibujo para su aplicación a un findeterminado. 2 - Introducir al alumno en el uso del instrumental y el conocimiento de las técnicas de medición ycálculo necesarias para realizar específicas de relevamiento planialtimétrico de hechos existentes, confecciónde planos topográficos y replanteo de obras.CONTENIDOS MÍNIMOS-PROGRAMA SINTÉTICO-Introducción. -Plano topográfico. Escalas. -Sistemas de Medición Regular. - Teoría de los errores. -Día nolaborable. -Teoría de los errores. -Errores, criterios de Gauss. -Errores, criterios de Chauvenet. -Video,Proyección. -Medición de ángulos con Teodolito. -Teodolito, El Anteojo, Sistemas de Graduación. -Medición deángulos verticales. -Medición directa de distancias. -Medición estadística de distancias. -Medición parláctica.Medición electróni